#Smart Nurse 간호기록 자동 생성 프로젝트

### 이프로젝트는...

## EDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cwd = '/content/drive/MyDrive/Colab Notebooks/Final_project/data/'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob

In [ ]:
# 데이터 불러오기

import json

with open(cwd + "nursingrecord_aws.json") as file :
  datas = json.load(file)

with open(cwd + "nursingrecord_ncp.json") as file :
  datas1 = json.load(file)

# 데이터프레임 생성
df = pd.DataFrame(datas) # aws
df1 = pd.DataFrame(datas1) # ncp



In [ ]:
patient = pd.read_csv(cwd + "patientinfo.csv")
patient

,patient_id,patient_name,patient_age,patient_gender,patient_height,patient_weight,patient_blood,patient_ward,patient_room,patient_department,...,student_college,patient_medical_note,vs_no,patient_pathology_no,patient_radiology_no,patient_medication_no,patient_create_date,patient_access_date,patient_nurse,patient_memo
0,1892264,마ㅇㅇ,70,2,170,87,A(+),서10,1035호,순환기내과,...,337.0,[입원동기]\n내원일 저녁 20시 30분경 목격된 arrest로 119 신고 후 R...,NaN,NaN,NaN,NaN,NaN,2023-09-14 09:23:44,NaN,NaN
1,2092147,홍옥X,77,1,151,52.6,O(+),21병동,47호실,신경외과,...,337.0,"[입원동기]\n22.09.02 본원에서 Spinal Fusion(Lumbar, Po...",NaN,NaN,NaN,NaN,NaN,2023-01-07 16:57:34,NaN,NaN
2,2140001,정수하(급성 호흡곤란 환아),0,2,49cm,"3,200g",O(+),소아병동,1호,소아청소년과,...,214.0,"※사례개요\n정수하 환아(F/0)는 2022년 12월 29일 10AM, 제왕 절개로...",NaN,NaN,NaN,NaN,NaN,2023-11-10 10:40:17,NaN,NaN
3,3230001,박영태(정맥수액주입)(인천가톨릭대),34,2,180cm,77kg,B(+),ER,ER-1,ER,...,323.0,인천가톨릭대 \n정맥주입 실습 기록용\n\n박영태(M/32) 님은 전일 철분결핍성 ...,NaN,NaN,NaN,NaN,NaN,2023-10-05 12:23:15,NaN,NaN
4,3360001,김영이,34,1,161,62,A+,산부인과,OBGY-02,OBGY,...,336.0,§ Patient’s Order §\n\nV/S check q 4hr\nBT che...,NaN,NaN,NaN,NaN,NaN,2023-10-25 17:04:38,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,9214000062,강쇼크(상황 발생 중),64,2,174,68,B(+),21병동,11호,외과,...,214.0,[사례 개요]\n본 사례는 Laparoscopic partial hepatectom...,NaN,NaN,NaN,NaN,NaN,2022-12-29 06:39:03,NaN,NaN
513,9214000063,강쇼크(상황 발생 후),64,2,174,68,B(+),21병동,11호,외과,...,214.0,[사례 개요]\n본 사례는 Laparoscopic partial hepatectom...,NaN,NaN,NaN,NaN,NaN,2022-12-29 06:39:49,NaN,NaN
514,9214000071,김한양(상황1),68,2,176,70,O(+),응급실,응급실,호흡기내과,...,214.0,"김한양(M/68) 환자는 3일전부터 sputum, cough 증상 발생했으며 1일 ...",NaN,NaN,NaN,NaN,NaN,2022-12-29 06:41:07,NaN,NaN
515,9214000072,김한양(상황2),68,2,176,70,O(+),응급실,응급실,호흡기내과,...,214.0,O₂ & nebulizer 치료 후 증상이 호전되어 경과 관찰을 하고 있던 중 모...,NaN,NaN,NaN,NaN,NaN,2022-12-29 06:41:47,NaN,NaN


In [ ]:
# 필요없는 컬럼 삭제
drops = ['nursing_record_id', 'user_id', 'update_at', 'record_time']
df = df.drop(drops, axis=1)
df1 = df1.drop(drops, axis=1)
df1

,patient_id,record_type,content,create_at
0,1445,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33
1,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58
2,1446,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55
3,1446,3,"{""narrativeNote"":""고체온. ""}",2022-08-15 12:26:28
4,1446,2,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo...",2022-08-15 15:18:40
...,...,...,...,...
7332,92000005,3,"{""narrativeNote"":""Moxicle 항생제 피부반응검사(AST)의 목적과...",2023-11-19 13:56:45
7333,92000005,3,"{""narrativeNote"":""AST 결과: negative. 주사부위 가려움, ...",2023-11-19 13:59:24
7334,92000023,3,"{""narrativeNote"":""기관절개관 삽입부위 발적, 부종 등과 같은 감염징후...",2023-11-19 14:05:52
7335,92000010,3,"{""narrativeNote"":""제왕절개술 준비를 위해 DR1 홍길동 처방에 의해 ...",2023-11-19 14:10:39


In [ ]:
# 오류찾기
b = df1[df1['record_type'] == 3]
b = b[~b['content'].str.contains('narrativeNote')]
b

,patient_id,record_type,content,create_at
1140,95000080,3,{},2023-03-06 03:59:09
2611,95000010,3,"{""assessment"":""이수정(M/50)님은 극심한 두통을 호소하며 응급실로 내...",2023-06-10 14:59:23
2620,95000080,3,"{""assessment"":""송진영(F/65)님은 외상성 지주막하 출혈(Traumat...",2023-06-10 16:03:58
2623,95000100,3,"{""assessment"":""권달순(F/28)님은 만성콩팥병(chronic renal...",2023-06-10 16:29:55
2625,95000110,3,"{""assessment"":""김영현(M/30)님은 4년 전 뇌경색(cerebral i...",2023-06-10 17:13:33
2974,92000014,3,{},2023-08-23 03:41:45
3032,92000014,3,{},2023-08-24 01:52:32
4486,92000002,3,{},2023-10-06 07:05:54
5471,92000007,3,{},2023-10-23 13:46:21
6773,92000002,3,{},2023-11-04 04:06:06


In [ ]:
# 오류찾기
b = df1[df1['record_type'] == 2]
b = b[~b['content'].str.contains('focus')]
b

,patient_id,record_type,content,create_at
2383,95000010,2,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn...",2023-06-02 18:05:42
3541,9770007,2,"{""data"":""3일간 대변활동 없어 불편 호소.\n“속이 불편해요.”\n"",""ac...",2023-09-06 05:33:01
3543,92000011,2,"{""data"":""\""3일간 대변을 보지 못해서 속이 더부룩하고 가스가 차요.\""\n...",2023-09-06 05:33:06
3544,9770007,2,"{""data"":""“3일동안 대변 못 봐서 속이 불편해요.”"",""action"":""의사...",2023-09-06 05:33:07
3546,93001020,2,"{""data"":""3일동안 대변을 보지 못해 속이 불편하다고 함."",""action"":...",2023-09-06 05:33:07
...,...,...,...,...
7271,92000007,2,"{""data"":""\""너무 어지러워서 움직일 수가 없어요\""\n-2일 전 위암으로 위...",2023-11-17 07:53:00
7272,92000012,2,"{""data"":""\""갑자기 가슴이 치밀어 오르는 듯이 아프고, 숨이 잘 안쉬어져요 ...",2023-11-17 07:59:53
7273,92000013,2,"{""data"":""\""숨쉬기가 너무 답답해요.\""\n-호흡곤란과 전신쇠약 증상으로 순...",2023-11-17 08:12:36
7318,92000012,2,"{""data"":""'숨쉬는 게 힘들어요'\nV/S BT 36.9 BP 130/90 ...",2023-11-18 11:51:39


In [ ]:
# 오류찾기
b = df[df['record_type'] == 3]
b = b[~b['content'].str.contains('narrativeNote')]
b

,patient_id,record_type,content,create_at
1047,2092147,3,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2023-01-13 03:11:37
1172,3370001,3,"{""domain"":""안정/보호"",""class"":""신체손상"",""diagnosis"":""...",2023-02-03 01:03:32
5456,63501070,3,{},2023-06-12 14:05:42
6601,95000120,3,{},2023-08-17 11:33:38
6938,63501020,3,{},2023-09-11 17:32:08
8047,3230001,3,{},2023-10-11 14:52:36
8109,93000025,3,"{""assessment"":""RR50 HR150 BT38.3 BP90/60\n환자는 ...",2023-10-12 03:28:23
8626,63501012,3,{},2023-10-20 13:07:15
9450,5410004,3,{},2023-10-23 23:41:42
10884,5410006,3,{},2023-10-25 20:30:13


record_type
- 0 : NANDA
- 1 : SOAPIE
- 2 : FOCUS DAR
- 3 : narrative note
- 4 : "cbe : Charting by Exception" 간호과정

In [ ]:
df['record_type'].value_counts() # 0 ~ 4 까지

3    9413
2    2016
0    1571
1     631
4     415
5       8
Name: record_type, dtype: int64

In [ ]:
df1['record_type'].value_counts()

3    5430
0     981
2     450
1     400
4      76
Name: record_type, dtype: int64

### aws, ncp 중복데이터 체크

In [ ]:
# aws, ncp 중복 데이터 체크

# 'patient_id' 컬럼을 기준으로 두 데이터프레임을 병합
merged_df = pd.merge(df, df1, on='patient_id', suffixes=('_aws', '_ncp'))

# 'content_df'과 'content_df1' 컬럼을 비교하여 'comparison' 컬럼 추가
merged_df['compa_content'] = merged_df['content_aws'] == merged_df['content_ncp']
merged_df['compa_rec_type'] = merged_df['record_type_aws'] == merged_df['record_type_ncp']
merged_df['compa_create'] = merged_df['create_at_aws'] == merged_df['create_at_ncp']

# 필요하다면 중복된 행을 제거
merged_df[['patient_id', 'compa_content', 'compa_rec_type', 'compa_create']].drop_duplicates()

merged_df



,patient_id,record_type_aws,content_aws,create_at_aws,record_type_ncp,content_ncp,create_at_ncp,compa_content,compa_rec_type,compa_create
0,1445,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,True,True,True
1,1445,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,False,False,False
2,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,False,False,False
3,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,True,True,True
4,1446,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55,True,True,True
...,...,...,...,...,...,...,...,...,...,...
899100,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2023-09-16 07:32:19,False,True,False
899101,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2023-10-30 08:03:55,False,True,False
899102,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,1,"{""data"":""너무 숨이 차다고 말로 표현함\n피로감 호소함\n\n4일간 체중 7...",2023-11-10 11:43:52,False,False,False
899103,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,1,"{""subjective"":""\""너무 숨이 차요!\"" 라고 호소함. 피로감을 호소함....",2023-11-12 10:14:26,False,False,False


In [ ]:
# 중복된 데이터 결과 출력
merged = merged_df[(merged_df['compa_content'] == True) & (merged_df['compa_rec_type'] == True) & (merged_df['compa_create'] == True)]
merged

,patient_id,record_type_aws,content_aws,create_at_aws,record_type_ncp,content_ncp,create_at_ncp,compa_content,compa_rec_type,compa_create
0,1445,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33,True,True,True
3,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58,True,True,True
4,1446,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55,True,True,True
8,1446,3,"{""narrativeNote"":""고체온. ""}",2022-08-15 12:26:28,3,"{""narrativeNote"":""고체온. ""}",2022-08-15 12:26:28,True,True,True
12,1446,2,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo...",2022-08-15 15:18:40,2,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo...",2022-08-15 15:18:40,True,True,True
...,...,...,...,...,...,...,...,...,...,...
872742,93000008,3,"{""narrativeNote"":""(전원)으로 옴 \nc/c : Dementia -...",2022-12-13 12:14:02,3,"{""narrativeNote"":""(전원)으로 옴 \nc/c : Dementia -...",2022-12-13 12:14:02,True,True,True
872755,93000008,3,"{""narrativeNote"":""귀중품 관리 및 도난 방지에 유의할 것을 설명함.\...",2022-12-13 12:16:45,3,"{""narrativeNote"":""귀중품 관리 및 도난 방지에 유의할 것을 설명함.\...",2022-12-13 12:16:45,True,True,True
873090,95000300,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2022-12-15 03:06:53,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2022-12-15 03:06:53,True,True,True
873154,93000013,3,"{""narrativeNote"":""20:30 환자 dyspnea 호소함.\nV/S:1...",2022-12-15 12:02:59,3,"{""narrativeNote"":""20:30 환자 dyspnea 호소함.\nV/S:1...",2022-12-15 12:02:59,True,True,True


In [ ]:
merged.to_csv('checking_3.csv', index=False, encoding='utf-8') # 중복된 데이터만 csv 저장해보기

In [ ]:
# ncp data : 중복된 데이터 제외하기
result_df1 = merged_df[(merged_df['compa_content'] == False) & (merged_df['compa_rec_type'] == True) & (merged_df['compa_create'] == False)]
result_df1

,patient_id,record_type_aws,content_aws,create_at_aws,record_type_ncp,content_ncp,create_at_ncp,compa_content,compa_rec_type,compa_create
14,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",2022-08-17 06:49:49,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2022-09-12 11:46:23,False,True,False
22,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",2022-08-17 06:49:49,0,"{""domain"":""대처/스트레스 내성"",""class"":""대처반응"",""diagnos...",2022-10-21 02:56:37,False,True,False
23,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",2022-08-17 06:49:49,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn...",2022-10-21 07:43:46,False,True,False
24,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",2022-08-17 06:49:49,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2022-11-17 07:37:33,False,True,False
25,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",2022-08-17 06:49:49,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn...",2022-11-17 07:40:23,False,True,False
...,...,...,...,...,...,...,...,...,...,...
899079,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ...",2023-05-17 04:22:21,False,True,False
899089,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없...",2023-06-04 02:29:42,False,True,False
899100,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2023-09-16 07:32:19,False,True,False
899101,90100001,0,"{""collectingData"":""sdsd"",""goal"":""sdsd"",""plan"":...",2023-11-16 23:51:45,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",2023-10-30 08:03:55,False,True,False


In [ ]:
drop_columns = ['content_aws', 'create_at_aws', 'record_type_aws', 'create_at_ncp', 'compa_content', 'compa_rec_type', 'compa_create']
result_df1 = result_df1.drop(drop_columns, axis=1)
result_df1

,patient_id,record_type_ncp,content_ncp
14,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
22,92000015,0,"{""domain"":""대처/스트레스 내성"",""class"":""대처반응"",""diagnos..."
23,92000015,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn..."
24,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
25,92000015,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn..."
...,...,...,...
899079,90100001,0,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ..."
899089,90100001,0,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없..."
899100,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
899101,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."


In [ ]:
# 중복된 데이터를 삭제한 ncp 데이터 최종본
result_df1 = result_df1.drop_duplicates(subset=['patient_id', 'content_ncp'], keep='first')
result_df1 = result_df1.reset_index()
result_df1.rename(columns={'record_type_ncp' : 'record_type',
                           'content_ncp' : 'content',
                           'index' : 'index'}, inplace=True)


,index,patient_id,record_type,content
0,14,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
1,22,92000015,0,"{""domain"":""대처/스트레스 내성"",""class"":""대처반응"",""diagnos..."
2,23,92000015,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn..."
3,24,92000015,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
4,25,92000015,0,"{""domain"":""활동/휴식"",""class"":""심혈관/호흡기계 반응"",""diagn..."
...,...,...,...,...
6682,898974,90100001,0,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ..."
6683,898984,90100001,0,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없..."
6684,898995,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
6685,898996,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."


In [ ]:
result_df1 = result_df1.drop('index', axis=1)

In [ ]:
# 오류찾기
a = result_df1[result_df1['record_type'] == 3]
a = a[~a['content'].str.contains('narrativeNote')]
a

,patient_id,record_type,content
937,92000002,3,{}
1417,92000007,3,{}
5310,95000080,3,{}
5360,95000080,3,"{""assessment"":""송진영(F/65)님은 외상성 지주막하 출혈(Traumat..."
5466,95000100,3,"{""assessment"":""권달순(F/28)님은 만성콩팥병(chronic renal..."
5547,95000110,3,"{""assessment"":""김영현(M/30)님은 4년 전 뇌경색(cerebral i..."
5780,95000010,3,"{""assessment"":""이수정(M/50)님은 극심한 두통을 호소하며 응급실로 내..."
6134,92000021,3,{}


In [ ]:
# aws 데이터 필요한 컬럼만 남기기
df.drop('create_at', axis=1)

,patient_id,record_type,content
0,1445,4,"{""cbe"":""Charting By Exception""}"
1,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라..."
2,1446,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""..."
3,1446,3,"{""narrativeNote"":""고체온. ""}"
4,1446,2,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo..."
...,...,...,...
14049,95000180,3,"{""narrativeNote"":""환자 SpO2 90%이며 쌕쌕거림을 확인하여 담당의..."
14050,5410003,3,"{""narrativeNote"":""11/17 \n10:30 NRS 7점으로 주치의에게..."
14051,92000003,3,"{""narrativeNote"":""11/17 \n10:30 NRS 7점으로 주치의에게..."
14052,95000040,3,"{""narrativeNote"":"" 9/25 17PM BST 260mg/dl 측정..."


In [ ]:
# 데이터프레임 concat
dff = pd.concat([df, result_df1], axis=0)
dff

,patient_id,record_type,content,create_at
0,1445,4,"{""cbe"":""Charting By Exception""}",2022-08-14 16:10:33
1,1445,1,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",2022-08-14 16:10:58
2,1446,0,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",2022-08-15 11:07:55
3,1446,3,"{""narrativeNote"":""고체온. ""}",2022-08-15 12:26:28
4,1446,2,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo...",2022-08-15 15:18:40
...,...,...,...,...
6682,90100001,0,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ...",NaN
6683,90100001,0,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없...",NaN
6684,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",NaN
6685,90100001,0,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",NaN


In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20741 entries, 0 to 6686
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   patient_id   20741 non-null  int64 
 1   record_type  20741 non-null  int64 
 2   content      20741 non-null  object
 3   create_at    14054 non-null  object
dtypes: int64(2), object(2)
memory usage: 810.2+ KB


### record type == 3 data (Narrative notes)

In [ ]:
# record type 3 table
t3 = dff[['patient_id', 'content']][dff['record_type'] == 3]
t3 = t3.reset_index()
t3

,index,patient_id,content
0,3,1446,"{""narrativeNote"":""고체온. ""}"
1,9,92000012,"{""narrativeNote"":""안ㄴ여알\nㄴ아ㅓ안ㅇ""}"
2,14,92000019,"{""narrativeNote"":""123321""}"
3,16,92000001,"{""narrativeNote"":""테스트 서술기록""}"
4,17,92000002,"{""narrativeNote"":""서술기록 테스트""}"
...,...,...,...
14521,6648,93001060,"{""narrativeNote"":""[통증 재사정] 통증 강도 NRS scale 5점\..."
14522,6649,93001060,"{""narrativeNote"":""0.45% DW 500ml 1P / 0.9% NS ..."
14523,6650,93001060,"{""narrativeNote"":""lab checked - Hb 9.8 low hc..."
14524,6651,93001060,"{""narrativeNote"":""MC Dr. 나의사 연락함""}"


In [ ]:
# 오류찾기
res = t3[~t3['content'].str.contains('narrativeNote')]
res

,index,patient_id,content,cleaned
459,1047,2092147,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",domain:안위class:신체적 안위diagnosis:급성 통증collecting...
549,1172,3370001,"{""domain"":""안정/보호"",""class"":""신체손상"",""diagnosis"":""...",domain:안정/보호class:신체손상diagnosis:낙상의 위험collecti...
2907,5456,63501070,{},
3829,6601,95000120,{},
4087,6938,63501020,{},
4598,8047,3230001,{},
4613,8109,93000025,"{""assessment"":""RR50 HR150 BT38.3 BP90/60\n환자는 ...",assessment:RR50 HR150 BT38.3 BP90/60 환자는 활동성이...
4849,8626,63501012,{},
5612,9450,5410004,{},
6908,10884,5410006,{},


In [ ]:
e = t3['content'][5]
e1 = t3['content'][426]
print(e)
print(e1)

{"narrativeNote":"1111"}
{"narrativeNote":"1/6 실습일지\n7:10  - C팀 1032, 1034호 환자들을 파악하고 아침 약 정리를 관찰하였다.\n7:40 - 경구투여를 관찰하였다.\n8:00~8:30 - 정규 V/S을 측정하였다. V/S 측정하며 필요한 것이 있으면 해드렸다.\n9:00 - 주사약 투여를 관찰하였다. 원래 plasma로 주는 환자 분이 계셨는데 어느 정도 기준의 output이 이루어지지 않고 있고 Sodium 수치가 높게 나타나 5% D/W로 변경한다고 설명해주셨다. 또한 lasix로 output을 유지하고 있다고 설명해주셨다.\n9:40 - DNR환자가 입원하셔서 보호자를 통해 간호정보조사지를 작성하는 것을 관찰하였다.\n10:00 - 1032호 3번 환자 분의 욕창 상태를 관찰하고 욕창 사진을 찍는 것을 관찰하였다. 보통 큰 욕창이 관찰되면 한개의 큰 드레싱 용품으로 욕창을 덮는데 현재는 드레싱 물품이 없어 2개의 드레싱 용품으로 욕창을 덮어두고 있다고 하셨다.\n10:30 - optiflow를 적용하고 있는 환자 분이 계셔서 물어보니 optiflow와 highflow가 같은 것이라고 설명해주셨다. 고유량 산소를 주입한다고 한다.\n11:00 - V-tach일 때는 전해질의 영향을 많이 받아 Mg 등의 수치를 조심해야 한다고 설명해주셨다. 따라서 단독 line으로 투여해야 한다.\n11:30 - 정규 V/S을 측정하였다.\n12:30 - 점심 식사 시간\n13:00 - 컨퍼런스"}


In [ ]:
import re

# 주어진 JSON 문자열 데이터
text_data = t3['content']

# JSON 문자열을 딕셔너리로 변환
# data_dict = eval(text_data)

# 필요한 텍스트 데이터 추출
# narrative_note = data_dict.get('narrativeNote')

def cleaned(text):
  # 특수문자 제거 및 줄바꿈 제거
  cleaned_text = re.sub(r'["",{}]', '', text, flags=re.UNICODE)
  cleaned_text = cleaned_text.replace('\n', ' ')
  cleaned_text = cleaned_text.replace('\\n', ' ')
  return cleaned_text

# 데이터프레임에 함수 적용
t3['cleaned'] = t3['content'].apply(cleaned)
t3


,index,patient_id,content,cleaned
0,3,1446,"{""narrativeNote"":""고체온. ""}",narrativeNote:고체온.
1,9,92000012,"{""narrativeNote"":""안ㄴ여알\nㄴ아ㅓ안ㅇ""}",narrativeNote:안ㄴ여알 ㄴ아ㅓ안ㅇ
2,14,92000019,"{""narrativeNote"":""123321""}",narrativeNote:123321
3,16,92000001,"{""narrativeNote"":""테스트 서술기록""}",narrativeNote:테스트 서술기록
4,17,92000002,"{""narrativeNote"":""서술기록 테스트""}",narrativeNote:서술기록 테스트
...,...,...,...,...
14521,6648,93001060,"{""narrativeNote"":""[통증 재사정] 통증 강도 NRS scale 5점\...",narrativeNote:[통증 재사정] 통증 강도 NRS scale 5점 - 통증...
14522,6649,93001060,"{""narrativeNote"":""0.45% DW 500ml 1P / 0.9% NS ...",narrativeNote:0.45% DW 500ml 1P / 0.9% NS 500m...
14523,6650,93001060,"{""narrativeNote"":""lab checked - Hb 9.8 low hc...",narrativeNote:lab checked - Hb 9.8 low hct 3...
14524,6651,93001060,"{""narrativeNote"":""MC Dr. 나의사 연락함""}",narrativeNote:MC Dr. 나의사 연락함


In [ ]:
t3.cleaned[300]

'narrativeNote:vaginal bleeding으로 타병원 내원하여 Pap smear 시행 중 squamous cell carcinoma 소견 있어 본원 OPD 내원 후squamous cell carcinoma 진단 하에  SCC of cervix stage IB2로 수술 위해 21병동 입원함.  dr. (39)에게 입원사실 알림. 담당 간호사를 소개하고 보호자에게 입원병동 안내함. 보호자에게 입원생활 안내문 제공하고 입원생활(병실구조 및 편의시설 환자권리보호 불편신고체계 면회제한 등)에 대하여 설명함. 보호자(기타)에게 영상정보수집 목적 설명하고 영상정보수집이용 동의서 받음. 지참약 복용안내문 이용하여 지참약 확인하고 주치의 처방하에 필요한 약 처방예정임을 설명함.'

### record type == 0 data (NANDA)

In [ ]:
t0 = dff[['patient_id', 'content']][dff['record_type'] == 0]
t0 = t0.reset_index()
t0

,index,patient_id,content
0,2,1446,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""..."
1,5,92000015,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만..."
2,11,92000001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만..."
3,12,92000001,"{""domain"":""건강증진"",""class"":""건강관리"",""diagnosis"":""비..."
4,13,92000019,"{""domain"":""자아지각"",""class"":""신체상"",""diagnosis"":""신체..."
...,...,...,...
2490,6682,90100001,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ..."
2491,6683,90100001,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없..."
2492,6684,90100001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."
2493,6685,90100001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급..."


In [ ]:
# 잘못된 데이터
string = 'narrativeNote'
result = t0[t0['content'].str.contains(string, case=False, na=False)]
# 결과 출력
print(result)


      index  patient_id                                            content
244     941    92000016                              {"narrativeNote":"s"}
261    1030     2092147  {"narrativeNote":"[간호일지]\n2023-01-05(목)\n-roun...
566    3208    92000007  {"narrativeNote":"<실습내용>\n-혈액투석(Hemodialysis) ...
710    5336    93000004  {"narrativeNote":"Suction 시 white + yellow 한 c...
843    7177    63501160  {"narrativeNote":"P: 왼쪽발목,  Q: 날카로운 느낌,  R: 움직...
1213   8615    43900001  {"narrativeNote":"환아 계속 보채어 까꿍놀이 하고 토닥이며 달래도록 ...
1246   9216     5410105  {"narrativeNote":"Epocelin 피부반응검사를 우측 전박내측에 실시...
1247   9219     5410105  {"narrativeNote":"AST결과 음성반응 보였으며 발적, 발열, 오심 등...
1248   9227     5410114  {"narrativeNote":"IV PCA 적용부위 부종, 혈종없이 잘 들어감. ...
1254   9342     5410004  {"narrativeNote":"두통과 갈증을 호소함. 혈당 측정 결과 180mg/...
1255   9368     5410114  {"narrativeNote":"수술 후 통증관리 를 위해 IV PCA에 대해 교육...
1294  10331     5410103  {"narrativeNote":"상기환자 복막염 수술 후 지속된 op site pa...
1295  10336     5410115  

In [ ]:
a = t0['content'][2]
a2 = t0['content'][1694]
print(a)
print(a2)

{"domain":"안위","class":"신체적 안위","diagnosis":"만성 통증","collectingData":"숨쉴때 통증 호소/NRS 6점 ","goal":"통증 경감/호흡을 정상적으로 할 수 있다.","plan":"통증 경감","interventions":"호흡시 통증 정도가 적은 자세를 취하도록함\n통증이 심할시 담당의사에게 알려 필요한 검사 또는 진통제를 투여함","evaluation":"호전됨"}
{"narrativeNote":"1/2 글리세린 50ml로 배출 관장 시행함. 관장 도중 경미한 복부 팽만감 호소하였으나 협조적임. 10분간 참았다가 대변을 보도록 하고, 배변 후 간호사에게 알려줄 것을 교육함.\n"}


In [ ]:
import re

# 주어진 JSON 문자열 데이터
text_data = t0['content']

# JSON 문자열을 딕셔너리로 변환
# data_dict = eval(text_data)

# 필요한 텍스트 데이터 추출
# narrative_note = data_dict.get('narrativeNote')

def cleaned(text):
  # 특수문자 제거 및 줄바꿈 제거
  cleaned_text = re.sub(r'["",{}]', '', text, flags=re.UNICODE)
  cleaned_text = cleaned_text.replace('\n', ' ')
  cleaned_text = cleaned_text.replace('\\n', ' ')
  return cleaned_text

# 데이터프레임에 함수 적용
t0['cleaned'] = t0['content'].apply(cleaned)
t0



,index,patient_id,content,cleaned
0,2,1446,"{""domain"":""안정/보호"",""class"":""체온조절"",""diagnosis"":""...",domain:안정/보호class:체온조절diagnosis:고체온collectingD...
1,5,92000015,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",domain:안위class:신체적 안위diagnosis:만성 통증collecting...
2,11,92000001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""만...",domain:안위class:신체적 안위diagnosis:만성 통증collecting...
3,12,92000001,"{""domain"":""건강증진"",""class"":""건강관리"",""diagnosis"":""비...",domain:건강증진class:건강관리diagnosis:비효율적 건강관리collec...
4,13,92000019,"{""domain"":""자아지각"",""class"":""신체상"",""diagnosis"":""신체...",domain:자아지각class:신체상diagnosis:신체상 혼란collecting...
...,...,...,...,...
2490,6682,90100001,"{""domain"":""영양"",""class"":""섭취"",""diagnosis"":""과체중의 ...",domain:영양class:섭취diagnosis:과체중의 위험collectingDa...
2491,6683,90100001,"{""collectingData"":""아직 외래에서 안 올라옴"",""goal"":""아직 없...",collectingData:아직 외래에서 안 올라옴goal:아직 없음plan:이분은...
2492,6684,90100001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",domain:안위class:신체적 안위diagnosis:급성 통증collecting...
2493,6685,90100001,"{""domain"":""안위"",""class"":""신체적 안위"",""diagnosis"":""급...",domain:안위class:신체적 안위diagnosis:급성 통증collecting...


### record type == 2 data (FOCUS DAR)

In [ ]:
t2 = dff[['patient_id', 'content']][dff['record_type'] == 2]
t2 = t2.reset_index()
t2

,index,patient_id,content
0,4,1446,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo..."
1,8,92000012,"{""focus"":""ㅇㄴㅇ\nㅇㄴㅇ"",""data"":""ㅇㄴㅇ\nㅇㄴㅇ\n"",""actio..."
2,27,92000007,"{""focus"":""복통"",""data"":""v/s:120/80-78-20-36.5"",""..."
3,31,92000003,"{""focus"":""배아파"",""data"":""v/s: 120/80-78-20-36\nP..."
4,39,92000019,"{""focus"":""1"",""data"":""1"",""action"":""1"",""response..."
...,...,...,...
2365,6453,9770007,"{""focus"":""3일동안 대변을 못봄"",""data"":""“속이 불편하고 더부룩해요”..."
2366,6454,9770007,"{""data"":""“3일 동안 배변을 못 봐서 가스가 차고 배가 아파요.”"",""act..."
2367,6455,9770007,"{""focus"":""환자의 정상적인 혈압과 호흡"",""data"":""혈압 210/120m..."
2368,6475,9770009,"{""focus"":""1"",""data"":""2"",""action"":""3"",""response..."


In [ ]:
c = t2['content'][2]
c2 = t2['content'][199]
print(c,'\n', c2)

{"focus":"복통","data":"v/s:120/80-78-20-36.5","action":"주치의한테 노티함\nprn 오더로 tridol 1A iv하자함\nIvinjec함","response":"복통없이 편안하다고함"} 
 {"focus":"-","data":"-폴리 카테터 유지 중임\n-소변 주머니에 있는 소변 색 탁함\n-혈액 검사 상 RBC, Urea Nitrogen, Creatine 상승되어 있음","action":"-감염 관리에 대해 교육함.  -무균술 실시함.\n-활력징후 측정함. -혈액검사와 소변검사 실시함.\n-처방된 약물 투여함.","response":"-활력징후 정상임\n-처방에 따라  levofloxacin hydrate OS(점안액) \nprn)[self] Cravit 0.5% 5ml oph sol [Drop] ins. 4/d 총 1btl 투여함\n-감염 증상 없음\n-개인 위생 관리 잘 하고 있음"}


In [ ]:
import re

# 주어진 JSON 문자열 데이터
text_data = t2['content']

# JSON 문자열을 딕셔너리로 변환
# data_dict = eval(text_data)

# 필요한 텍스트 데이터 추출
# narrative_note = data_dict.get('narrativeNote')

def cleaned(text):
  # 특수문자 제거 및 줄바꿈 제거
  cleaned_text = re.sub(r'["",{}]', '', text, flags=re.UNICODE)
  cleaned_text = cleaned_text.replace('\n', ' ')
  cleaned_text = cleaned_text.replace('\\n', ' ')
  return cleaned_text

# 데이터프레임에 함수 적용
t2['cleaned'] = t2['content'].apply(cleaned)
t2


,index,patient_id,content,cleaned
0,4,1446,"{""focus"":""12"",""data"":""12"",""action"":""12"",""respo...",focus:12data:12action:12response:12
1,8,92000012,"{""focus"":""ㅇㄴㅇ\nㅇㄴㅇ"",""data"":""ㅇㄴㅇ\nㅇㄴㅇ\n"",""actio...",focus:ㅇㄴㅇ ㅇㄴㅇdata:ㅇㄴㅇ ㅇㄴㅇ action:ㅇㄴ ㄴㅇ respons...
2,27,92000007,"{""focus"":""복통"",""data"":""v/s:120/80-78-20-36.5"",""...",focus:복통data:v/s:120/80-78-20-36.5action:주치의한테...
3,31,92000003,"{""focus"":""배아파"",""data"":""v/s: 120/80-78-20-36\nP...",focus:배아파data:v/s: 120/80-78-20-36 POD 2일째 pca...
4,39,92000019,"{""focus"":""1"",""data"":""1"",""action"":""1"",""response...",focus:1data:1action:1response:1
...,...,...,...,...
196,655,9214000021,"{""focus"":""복부 통증"",""data"":""장음이 약하고 복부팽창 있음."",""ac...",focus:복부 통증data:장음이 약하고 복부팽창 있음.action:환자상태 사정...
197,660,9214000021,"{""focus"":""CT 설명함."",""data"":""X-ray 검사 상 병변 잘 확인되...",focus:CT 설명함.data:X-ray 검사 상 병변 잘 확인되지 않아서 CT ...
198,699,92000007,"{""focus"":"" "",""data"":""-혈액검사 결과 감염 위험 지표 있음 \n-...",focus: data:-혈액검사 결과 감염 위험 지표 있음 -POD3일째임 -두...
199,720,92000010,"{""focus"":""-"",""data"":""-폴리 카테터 유지 중임\n-소변 주머니에 있...",focus:-data:-폴리 카테터 유지 중임 -소변 주머니에 있는 소변 색 탁함 ...


In [ ]:
t2_data = t2['content']

# Series의 각 원소를 문자열로 변환
t2_data_str = t2_data.astype(str)

# 빈 데이터프레임 생성
type_2 = pd.DataFrame()

# Series의 각 원소에 대해 JSON 데이터를 파싱하고 데이터프레임에 추가
for json_str in t2_data_str:
    try:
        # JSON 데이터 파싱
        data_dict = json.loads(json_str)

        # 데이터프레임에 추가
        type_2 = pd.concat([type_2, pd.DataFrame([data_dict])], ignore_index=True)

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
type_2



,focus,data,action,response,narrativeNote,domain,class,diagnosis,collectingData,goal,plan,interventions,evaluation
0,12,12,12,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ㅇㄴㅇ\nㅇㄴㅇ,ㅇㄴㅇ\nㅇㄴㅇ\n,ㅇㄴ\nㄴㅇ\n,ㄴㅇ\nㄴㅇ\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,복통,v/s:120/80-78-20-36.5,주치의한테 노티함\nprn 오더로 tridol 1A iv하자함\nIvinjec함,복통없이 편안하다고함,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,배아파,v/s: 120/80-78-20-36\nPOD 2일째\npca 유지중\nNRS 7점,tridol 1@ IM injected by RN 김성국,NRS 2점 관찰됨\nv/s : 120/ 80-80-20-36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,3일동안 대변을 못봄,“속이 불편하고 더부룩해요”,Glycerin enema 50cc,“변을 봤어요”\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2366,NaN,“3일 동안 배변을 못 봐서 가스가 차고 배가 아파요.”,의사처방 하에 글리세린 enema 5cc 시행함,“속이 편안해졌어요”,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2367,환자의 정상적인 혈압과 호흡,"혈압 210/120mmHg, 분당 호흡수 32회, 요흔성 부종",2L/분으로 nasal cannula를 이용해 산소투여함\n칼륨소모성 이뇨제 La...,혈압 150/100 으로 감소함\n호흡수 20회/분으로 감소함\npitting ed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2368,1,2,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def cleaned(text):
  # 특수문자 제거 및 줄바꿈 제거
  cleaned_text = re.sub(r'["",{}]', '', text, flags=re.UNICODE)
  cleaned_text = cleaned_text.replace('\n', ' ')
  cleaned_text = cleaned_text.replace('\\n', ' ')
  return cleaned_text

# 데이터프레임에 함수 적용
type_2 = type_2.apply(cleaned)
type_2

UnboundLocalError: ignored

### record type == 1 data (SOAPIE)

In [ ]:
t1 = dff[['patient_id', 'content']][dff['record_type'] == 1]
t1 = t1.reset_index()
t1

,index,patient_id,content
0,1,1445,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라..."
1,6,92000012,"{""subjective"":""안ㄴ여\n안영\n"",""objective"":""ㅇㄴㅇ"",""a..."
2,7,92000012,"{""subjective"":""ㅂ. 인ㄴ\n2,ㅇ\n\n3.\n3\nㅇ"",""object..."
3,37,92000019,"{""subjective"":""1"",""objective"":""1"",""assessment""..."
4,45,92000009,"{""subjective"":""통증"",""objective"":""통증"",""assessmen..."
...,...,...,...
868,6445,9770007,"{""subjective"":""외상은 보이지 않으나 어지럼증 호소"",""objective..."
869,6446,9770007,"{""subjective"":""너무 숨이 차다고 호소.\n피로감 호소."",""object..."
870,6447,9770007,"{""focus"":""환자의 정상적인 혈압과 호흡"",""data"":""혈압 210/120m..."
871,6448,9770007,"{""subjective"":""숨이 너무 차요."",""objective"":""호흡하는 것이..."


In [ ]:
d = t1['content'][0]
d2 = t1['content'][92]
print(d)
print(d2)

{"subjective":"비브리오 콜레라","objective":"비브리오 콜레라","assessment":"비브리오 콜레라","planning":"비브리오 콜레라","interventions":"비브리오 콜레라","evaluation":"비브리오 콜레라"}
{"subjective":"어지러워요. 기운이 없어요. 몸이 쳐져요. \n숨 쉬는거 힘들어요. 기침 자주 나와요.","objective":"고열. 고열로 인한 열성 경련. 호흡기세포융합바이러스로 인한 급성 기관지염.","assessment":"없음.\n","planning":"산소줄 끼고 있는데도 포화도 자꾸 떨어져서\n산소 쎄게 틀어주었고 그래도 포화도 90 이하로 떨어지면 chest CT 찍기로 함.","interventions":"없음.","evaluation":"없음."}


In [ ]:
import re

# 주어진 JSON 문자열 데이터
text_data = t1['content']

# JSON 문자열을 딕셔너리로 변환
# data_dict = eval(text_data)

# 필요한 텍스트 데이터 추출
# narrative_note = data_dict.get('narrativeNote')

def cleaned(text):
  # 특수문자 제거 및 줄바꿈 제거
  cleaned_text = re.sub(r'["",{}]', '', text, flags=re.UNICODE)
  cleaned_text = cleaned_text.replace('\n', ' ')
  cleaned_text = cleaned_text.replace('\\n', ' ')
  return cleaned_text

# 데이터프레임에 함수 적용
t1['cleaned'] = t1['content'].apply(cleaned)
t1


,index,patient_id,content,cleaned
0,1,1445,"{""subjective"":""비브리오 콜레라"",""objective"":""비브리오 콜레라...",subjective:비브리오 콜레라objective:비브리오 콜레라assessmen...
1,6,92000012,"{""subjective"":""안ㄴ여\n안영\n"",""objective"":""ㅇㄴㅇ"",""a...",subjective:안ㄴ여 안영 objective:ㅇㄴㅇassessment:ㄴㅇpl...
2,7,92000012,"{""subjective"":""ㅂ. 인ㄴ\n2,ㅇ\n\n3.\n3\nㅇ"",""object...",subjective:ㅂ. 인ㄴ 2ㅇ 3. 3 ㅇobjective:ㅇassessme...
3,37,92000019,"{""subjective"":""1"",""objective"":""1"",""assessment""...",subjective:1objective:1assessment:11planning:1...
4,45,92000009,"{""subjective"":""통증"",""objective"":""통증"",""assessmen...",subjective:통증objective:통증assessment:통증planning...
...,...,...,...,...
89,848,92000002,"{""subjective"":""가래가 많이나와요"",""objective"":""객담 배출 여...",subjective:가래가 많이나와요objective:객담 배출 여러차례 기침 자주...
90,879,93000007,"{""subjective"":""기운이 없어요. 목이 아파요. 기침이 나와요.\n숨 쉬기...",subjective:기운이 없어요. 목이 아파요. 기침이 나와요. 숨 쉬기 힘들어요...
91,887,92000016,"{""subjective"":""기력이 딸리고 양쪽 hip pain 호소(OS f/u) ...",subjective:기력이 딸리고 양쪽 hip pain 호소(OS f/u) 걸을때힘...
92,890,93000007,"{""subjective"":""어지러워요. 기운이 없어요. 몸이 쳐져요. \n숨 쉬는거...",subjective:어지러워요. 기운이 없어요. 몸이 쳐져요. 숨 쉬는거 힘들어요...


### record type == 4 (간호과정)

In [ ]:
t4 = dff[['patient_id', 'content']][dff['record_type'] == 4]
t4 = t4.reset_index()
t4